In [1]:
category = [
        'comics',
        'not-interested',
        'humor',
        'horror',
        'adventure',
        'tragedy',
]

In [2]:
!pip install requests_html

     |████████████████████████████████| 153kB 3.3MB/s 
     |████████████████████████████████| 133kB 4.1MB/s 
     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=46a57b0956b739c3e9151da1171ea6d7385d4875c5b43f6b4e47d703269534e7
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24133 sha256=6d473638072a1ebdf237f531125067e2205d5d6c8e996bfbd1caa75bdc39d9ed
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
Successfully built fake-useragent parse
ERROR: kaggle 1.5.8 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.10 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is i

In [3]:
import multiprocessing
import threading
import csv
import pandas as pd
from bs4 import BeautifulSoup as bs
from requests_html import HTMLSession
import random as rand
from urllib.parse import urljoin
from time import *

session = HTMLSession()

def init(l):
    global lock
    lock = l

def scrape(urls):
    l_data = []
    set_no = urls[1]
    urls = urls[0]
    count = 1
    for url in urls:
        print(set_no, count, url)
        page = session.get(url)
        soup = bs(page.html.html, 'html.parser')
        titles = soup.find_all('a', class_='bookTitle')
        authors = soup.find_all('a', class_='authorName')

        for title, author in zip(titles, authors):
            try:
                bookURL = "https://www.goodreads.com" + title["href"]
                # print(bookURL)
                page = session.get(bookURL)
                soup = bs(page.html.html, 'html.parser')
                imageLink = soup.find_all("img", id = 'coverImage')[0]["src"]
                # print('got image link')
                summary = soup.find_all('div', class_ = 'readable stacked')[0]
                # print('got summary')
                temp = summary.findChildren("span", recursive=False)
                # print('got summary')
                if(len(temp)>1):
                    bookSummary = temp[1].get_text()
                else:
                    bookSummary = temp[0].get_text()
                bookFormat = soup.find_all('span', itemprop = 'bookFormat')[0].get_text()
                # print('got format')
                yearPublished = soup.find_all('div', id = 'details')[0].findChildren("div")[1].get_text()
                # print('got publish')
                noOfPages = soup.find_all('span', itemprop = 'numberOfPages')[0].get_text()
                # print('got pages')
                rating = soup.find_all('span', itemprop = 'ratingValue')[0].get_text()
                # print('got ratings')
                ratingCount = soup.find_all('meta', itemprop = 'ratingCount')[0]["content"]
                # print('got rating count')
                reviewCount = soup.find_all('meta', itemprop = 'reviewCount')[0]["content"]
                # print('got review count')
                language = soup.find_all('div', itemprop = 'inLanguage')[0].get_text()
                # print('got language')
                noOfAwards = soup.find_all('div', itemprop = 'awards')
                # print('got awards')
                if(noOfAwards):
                    noOfAwards = len(noOfAwards[0].find_all('a', class_ = "award"))
                else:
                    noOfAwards = 0
                authorFollowers = soup.find_all('div', class_="bookAuthorProfile__followerCount")[0].get_text()
                # print('got author followers')
                try:
                    isbn = soup.find_all('span', itemprop = 'isbn', recursive = True)[0].get_text()
                except:
                    try:
                        isbn = soup.find_all('div', itemprop = 'isbn')[0].get_text()
                    except:
                        isbn = soup.find_all('infoBoxRowItem', itemprop = 'isbn')[0].get_text()
                # print('got isbn')
                genre = soup.find_all('a', class_ = "actionLinkLite bookPageGenreLink")
                # print('got genre')
                genre = [node.get_text() for node in genre]
                genre = "//".join(i for i in genre)
                # print('got genre')
                try:
                    l_data.append([isbn, title.get_text(), author.findChildren('span')[0].get_text(), imageLink, bookSummary, bookFormat, yearPublished, genre, noOfPages, rating, ratingCount, reviewCount, language, noOfAwards, authorFollowers])
                except:
                    l_data.append([isbn, title.findChildren('span')[0].get_text(), author.findChildren('span')[0].get_text(), imageLink, bookSummary, bookFormat, yearPublished, genre, noOfPages, rating, ratingCount, reviewCount, language, noOfAwards, authorFollowers])
                # print([isbn, title.findChildren('span')[0].get_text(), author.findChildren('span')[0].get_text(), imageLink, bookSummary, bookFormat, yearPublished, genre, noOfPages, rating, ratingCount, reviewCount, language, noOfAwards, authorFollowers])
                # print('book', set_no, len(l_data))
            except Exception as e:
                # print(e)
                # print('error')
                continue
        lock.acquire()
        df = pd.DataFrame(l_data)
        df.columns = ['isbn', 'title', 'author', 'imageLink', 'bookSummary', 'bookFormat', 'yearPublished', 'genre', 'noOfPages', 'rating', 'ratingCount', 'reviewCount', 'language', 'noOfAwards', 'authorFollowers']
        fileName = str(set_no) + ".csv"
        df.to_csv(fileName, mode = 'a')
        l_data = []
        count += 1
        lock.release()

In [4]:
# lock = threading.Lock() 
baseUrl = "https://www.goodreads.com/search?q="
baseUrl2 = "&qid=Nn68gjHZeL&tab=books&page="
all_urls = []
# for set_no in range(len(category)):
for set_no in 'ijklmnop':
    urls = []
    for page_no in range(1, 101):
        urls.append(baseUrl + set_no + baseUrl2 + str(page_no))
    all_urls.append([urls, set_no])
print(all_urls)

l = multiprocessing.Lock()
p = multiprocessing.Pool(processes=8, initializer=init, initargs=(l,))
p.map(scrape, all_urls)
p.close()
p.join()

[[['https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=1', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=2', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=3', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=4', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=5', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=6', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=7', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=8', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=9', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=10', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=11', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=12', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=books&page=13', 'https://www.goodreads.com/search?q=i&qid=Nn68gjHZeL&tab=